In [1]:
import pandas as pd 
import re
import requests
import pickle

In [2]:
data = requests.get('https://www.ams.usda.gov/mnreports/jo_gr112.txt').text
pickle.dump(data,open('close_315.pickle','wb'))
# data2 = data.split('\r\n')
# data2
data = pickle.load(open('close.pickle','rb'))


In [3]:
# res = re.findall(r'(^\w{3}=.$)',data,re.MULTILINE)
# res
crops= [    
            'Louisiana Midday bids and basis for US 2 Soft Red Winter Wheat',
            'Louisiana Midday bids and basis for US 2 Yellow Corn',
            'Louisiana Midday bids and basis for US 1 Yellow Soybeans',
            'Louisiana Midday bids and basis for US 2 Yellow Sorghum',
            'Texas Midday bids and basis for US 1 Hard Red Winter Wheat',
            'Texas Midday bids and basis for US 2 Yellow Sorghum'
       ]


r = re.compile(r'|'.join(crops))
reports = re.split(r,data) #splits based on crop
reps = [x.split('\r\n') for x in reports]


reps



[['',
  'JO_GR112',
  'Portland, OR    Mon  Mar 18, 2019    USDA Market News',
  '',
  'Louisiana and Texas Export Bids',
  '',
  'Gulf Export bids and basis for grain delivered to Gulf export elevators,',
  'CIF barge to Louisiana Gulf (Mississippi River) and rail-truck Texas Gulf,',
  'delivery periods as specified, dollars per bushel, except sorghum per cwt.',
  'Bids as of 11:30 Central time; Subject to change.',
  '',
  '   In early trading bids were not fully established but were indicated as',
  'follows for Louisiana:  soft red winter wheat lower, yellow corn lower,',
  'soybeans were lower and yellow sorghum was not available. Texas cash bids',
  'were lower for hard red winter wheat and lower for yellow sorghum.',
  '',
  '',
  'Louisiana Morning bids and basis for US 2 Soft Red Winter Wheat',
  'Cash Bids                           Basis',
  'Mar=    5.4325-5.4825                +87K  to +92K',
  'Apr=    5.4125-5.4225                +85K  to +86K',
  'May=    5.3925-5.4225  

In [4]:
list1= []

a = re.compile(r'^\w{,3}=')

for x in reps[:]:
    new = [ y for y in x if re.match(a,y) ]
    list1.append(new)


[['Mar=    5.4325-5.4825                +87K  to +92K',
  'Apr=    5.4125-5.4225                +85K  to +86K',
  'May=    5.3925-5.4225                +83K  to +86K',
  'Jun=    5.2825-5.3125                +65N  to +68N',
  'Jul=    5.2825-5.3125                +65N  to +68N',
  'Aug=    NA                             NA',
  'Mar=    4.1775-4.2575                +47K  to +55K',
  'Apr=    4.1675-4.2375                +46K  to +53K',
  'May=    4.1675-4.2375                +46K  to +53K',
  'Jun=    4.2500                       +45N',
  'Jul=    4.2500                       +45N',
  'Aug=    4.2975                       +43U',
  'Mar=    9.2325-9.5125                +21K  to +49K',
  'Apr=    9.2125-9.4525                +19K  to +43K',
  'Jun=    9.3475-9.5475                +19N  to +39N',
  'Jul=    9.3875-9.5475                +23N  to +39N',
  'Aug=    NA                             NA',
  'Mar=    NA                             NA',
  'Mar=    5.4800-5.6600               +112K  

In [5]:
def get_crop_type():
    crop_count = []
    for item in list1:
        if item  !=[]:
            crop_count.append(len(item))
    crops_list = []
    for x in range(len(crop_count)):
        crops_list.append( [crops[x] for crop in range(0,crop_count[x])])

    crop_list_final = [item for sublist in crops_list for item in sublist]
    return crop_list_final


crop_type = get_crop_type() # this give the right column for the data frame

In [6]:
list3= []
for x in list1:
    for y in x:
        list3.append(re.split(r'\s{2,}',y)) 


In [7]:
# cash bids     change   basis     change     
df3= pd.DataFrame(list3,columns=['Month','Cash_Bids', 'Change1', 'Basis1_og', 'Change2'])


try:
    df3['Basis1_1'] = df3['Basis1_og'].str.split('to',expand= True)[0].str.replace('\D','')
    df3['Basis1_2'] = df3['Basis1_og'].str.split('to',expand= True)[1].str.replace('\D','')
except KeyError:
    df3['Basis1_1'] = df3['Basis1_og']
    df3['Basis1_2'] = None


try:
    df3['Change2_1'] = df3['Change2'].str.split('-',expand=True)[0]
    df3['Change2_2'] = df3['Change2'].str.split('-',expand=True)[1]
except KeyError:
    df3['Change2_1'] = df3['Change2']
    df3['Change2_2'] = None

try:
    df3['Change1_1'] = df3['Change1'].str.split(' ',expand=True)[0]
    df3['Change1_2'] = df3['Change1'].str.split(' ',expand=True)[1]
except:
    df3['Change1_1'] = df3['Change1']
    df3['Change1_2'] = None

df3['Crop'] = crop_type 

df3['Date'] = pd.Timestamp.now() #this will be fine
df3['Cash_Bids_1']= df3['Cash_Bids'].str.split('-', expand=True)[0]
df3['Cash_Bids_2'] =df3['Cash_Bids'].str.split('-', expand=True)[1]

df3['Contract'] = df3['Basis1_1'].str.replace('\d','') 
df3['Month']= df3['Month'].str.strip('=')
df3

close = df3[['Month','Cash_Bids_1','Cash_Bids_2','Change1_1','Change1_2', 'Basis1_1','Basis1_2', 'Contract','Change2_1','Change2_2','Crop','Date' ]]
close = close.fillna('NA')
close

AssertionError: 5 columns passed, passed data had 4 columns

In [ ]:
pickle.dump (close,open('jogr112_close_frame','wb'))
